In [1]:
import numpy as np
from qiskit import *
from qiskit import Aer
from qiskit.aqua.components.optimizers import *
from qiskit.quantum_info import state_fidelity
from qiskit.quantum_info.analysis import average_data,hellinger_fidelity
from qiskit.providers.aer.noise import NoiseModel
from qiskit.aqua.utils import get_subsystem_density_matrix
np.set_printoptions(precision=15)
%matplotlib inline

In [2]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright


In [ ]:
provider = IBMQ.load_account()
provider.backends()
device_backend = provider.get_backend('ibmq_vigo')
coupling_map = device_backend.configuration().coupling_map
noise_model = NoiseModel.from_backend(device_backend)
print(noise_model)
basis_gates = noise_model.basis_gates

# Select the QasmSimulator from the Aer provider
simulator = Aer.get_backend('qasm_simulator')

In [ ]:
def circuit(params):
    q = QuantumRegister(2)
    c = ClassicalRegister(2)
    qc = QuantumCircuit(q,c)
    #qc.h(q[0])
    
    qc.ry(params[0],q[0])
    qc.rx(params[1],q[0])
    
    qc.cx(q[0],q[1])
    qc.snapshot('0')
    qc.measure(q[0],c[0])
    qc.measure(q[1],c[1])
    job = execute(qc, simulator, 
                       noise_model=noise_model,
                       coupling_map=coupling_map,
                       basis_gates=basis_gates,shots=100,backend_options={"max_parallel_threads":10}).result()
    
    statevector = np.array(job.data()['snapshots']['statevector']['0'][0])
    statevector = get_subsystem_density_matrix(statevector,[2,3,4])
    counts = job.get_counts()
    return counts, statevector
    

In [ ]:
def cross_entropy(pred, targets, epsilon=1e-20):

    pred = np.clip(pred, epsilon, 1. - epsilon)
    
    n_samples = pred.shape[0]
    
    loss = -np.sum(targets*np.log10(pred+epsilon))/n_samples

    return loss

labels = np.array([0.5,0,0,0.5])
labels_vector = np.array([1/np.sqrt(2),0,0,1/np.sqrt(2)])
p = []
c = {'01': 0, '11': 50, '00': 50, '10': 0}
def cost(var):
    

#    print(var)
    st_vec = circuit(var)[1]
#    print(st_vec)
#    return cross_entropy(circuit(var),la)
    
    res = abs(1 - state_fidelity(st_vec,labels_vector))
    #res = - hellinger_fidelity(c,st_vec)
#    print(hellinger_fidelity(c,st_vec))
#    print(res)
    #if res < loss:
 #   print ('hey ',var)
#        loss = res
    return res

In [ ]:
np.random.seed(5)
init_params = np.random.randn(2)
#init_params = [1.57079633]
print(init_params)
print(cost(init_params))

In [ ]:
opt = AQGD(disp=True,maxiter=100)
#opt = ADAM(tol=1e-11,lr=0.05)

In [ ]:
opt.optimize(objective_function=cost,num_vars=2,initial_point=init_params)

In [ ]:
np.pi/2

In [ ]:
circuit([-6.1816324548196375])[0]

In [ ]:
1024/2

In [ ]:
x1 = np.linspace(-1, 2, 60)
x2 = np.linspace(-1, 2, 60)
X1, X2 = np.meshgrid(x1, x2)
print(x1.shape)
Z = np.zeros((len(X2),len(X1)))

for id0,i in enumerate(x1):
    for id1,j in enumerate(x2):
        Z[id0,id1] = cost([i,j])

In [ ]:
circuit([x1[0],x2[20]])

In [ ]:
levels = [0.0, 1.0, 2.0, 4.0, 8.0, 12.0, 14.0]
cp = plt.contour(X1, X2, Z, colors='black', linestyles='dashed', linewidths=1)
plt.clabel(cp, inline=3, fontsize=10)
cp = plt.contourf(X1, X2, Z,cmap=plt.cm.hot)
plt.colorbar()
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

In [ ]:
cm = plt.cm.get_cmap('viridis')
plt.scatter(X1, X2, c=Z, cmap=cm)
plt.colorbar()
plt.show()